In [1]:
# Rev Apr 2021
# By Zhijing Eu (zhijingeu@yahoo.com)
# This notebook demonstrates how to use a Brute Force Search method to 
# solve the Wedding Seating Arrangement Problem

In [2]:
import pandas as pd
import numpy as np
import random
import math
import functools
import operator
import networkx as nx

In [3]:
InputFileName="Example_Guests96pax_TableSize8pax.xls"

GuestListRaw = pd.read_excel(InputFileName, 'GuestList')
# RelMatrixRawList=RelMatrixRaw.values.tolist()
GuestList=GuestListRaw["Guest"].values.tolist()

In [4]:
GuestList

['Ms.Annikki Pietikäinen',
 'Ms.Line Kristiansen',
 'Mr.Koppány Baranyi',
 'Ms.Yukiko Koyama',
 'Ms.Johanna Wechsler',
 'Mr.Kristof Kuntz',
 'Ms.Živana Breceljnik',
 'Ms.Jitakatefuari ',
 'Mr.Bao Chin',
 'Mrs.Bikatu Bataev',
 'Mr.Teppo Lydman',
 'Ms.Adabella Quezada',
 'Ms.Apolonia Kucharska',
 'Ms.Lundy Moquin',
 'Mr.Jonathan Jessen',
 'Mrs.Jonna Parras',
 'Mr.Eduardo Santos',
 'Mr.Joao Ribeiro',
 'Mr.Meikuaritakufu ',
 'Mr.Géza Kovács',
 'Mr.Uzor Pál',
 'Mr.Þorgrímur Búason',
 'Mr.Shamsuddi Desheriyev',
 'Ms.Orzsebet Lendvai',
 'Mrs.Sarah Mortensen',
 'Ms.Miranda Zaragamba',
 'Mrs.Huan Yue Shen',
 'Mr.Cicerón Ozuna',
 'Mr.Shikimirikashite ',
 'Mr.Loren Pullman',
 'Mrs.Chinyere Onwuka',
 'Ms.Quinzia Schiavone',
 'Mr.Ross Morrison',
 'Mr.Yasser Hakimi',
 "Mr.Leal D'Aoust",
 'Dr.Hólmfríður Örlygsdóttir',
 'Mr.Khalil Alonso',
 'Mrs.Tiina Johanson',
 'Ms.Алла Евдокимова',
 'Mrs.Yobachi Chukwueloka',
 'Mr.Satish Muntslag',
 'Ms.Franziska Beich',
 'Mrs.Lia Göransson',
 'Mrs.Zuzanna McIntosh

In [5]:
RelMatrixRaw=GuestListRaw.dropna(thresh=2)
RelMatrixRaw

,Guest,Together1,Together2,Together3,Apart1,Apart2,Apart3
0,Ms.Annikki Pietikäinen,Ms.Line Kristiansen,NaN,NaN,NaN,NaN,NaN
4,Ms.Johanna Wechsler,Mr.Kristof Kuntz,Ms.Živana Breceljnik,NaN,NaN,NaN,NaN
8,Mr.Bao Chin,Mrs.Bao Chin,NaN,NaN,Ms.Miranda Zaragamba,NaN,NaN
22,Mr.Shamsuddi Desheriyev,NaN,NaN,NaN,Ms.Orzsebet Lendvai,NaN,NaN
23,Ms.Orzsebet Lendvai,NaN,NaN,NaN,Mr.Shamsuddi Desheriyev,NaN,NaN
25,Ms.Miranda Zaragamba,NaN,NaN,NaN,Mr.Bao Chin,NaN,NaN
30,Mrs.Chinyere Onwuka,Ms.Quinzia Schiavone,NaN,NaN,NaN,NaN,NaN
31,Ms.Quinzia Schiavone,Mr.Ross Morrison,NaN,NaN,NaN,NaN,NaN
32,Mr.Ross Morrison,Mr.Yasser Hakimi,Mr.Leal D'Aoust,NaN,NaN,NaN,NaN
33,Mr.Yasser Hakimi,NaN,NaN,NaN,Dr.Hólmfríður Örlygsdóttir,NaN,NaN


In [6]:
RelMatrixRawList=RelMatrixRaw.values.tolist()

RelGuest=[]
RelConditionsTogether=[]
RelConditionsApart=[]
for element in RelMatrixRawList:
    RelGuest.append(element[0])
    RelConditionsTogether.append([element[1],element[2],element[3]])
    RelConditionsApart.append([element[4],element[5],element[6]])


In [7]:
noOfGuests=len(GuestList)
table_size = pd.read_excel(InputFileName, 'TableSize').columns[1]
table_count = len(GuestList) // table_size
if len(GuestList)%table_size==0:
    blank_seats=[]
if len(GuestList)%table_size!=0:
    blank_seats=list(range(1,((table_count+1)*table_size-len(GuestList))+1))
    blank_seats=["Spare-"+str(x) for x in blank_seats]
GuestList=GuestList+blank_seats
len(GuestList)
noOfGuests=len(GuestList)
print(noOfGuests)

96


In [8]:
RandomArrangement=[]

print("No Of Guests=",str(noOfGuests))
print("Table Size=",str(table_size))
print("Required No Of Tables=",str(int(noOfGuests/table_size)))

No Of Guests= 96
Table Size= 8
Required No Of Tables= 12


In [9]:
#Generates a random arrangement as "Seed" for simulation
for i in range(1,int(noOfGuests/table_size)+1):
    RandomArrangement.extend([i]*table_size)
random.shuffle(RandomArrangement)
RandomArrangement

[5,
 2,
 5,
 9,
 9,
 6,
 7,
 4,
 1,
 12,
 12,
 10,
 10,
 1,
 5,
 12,
 4,
 6,
 8,
 11,
 12,
 2,
 11,
 1,
 10,
 3,
 3,
 4,
 2,
 12,
 6,
 8,
 3,
 7,
 9,
 3,
 11,
 2,
 3,
 1,
 6,
 1,
 10,
 4,
 5,
 5,
 4,
 7,
 2,
 12,
 7,
 8,
 7,
 5,
 9,
 12,
 2,
 11,
 6,
 6,
 9,
 9,
 8,
 3,
 10,
 7,
 11,
 3,
 4,
 11,
 7,
 2,
 9,
 1,
 11,
 4,
 7,
 1,
 9,
 8,
 6,
 1,
 11,
 8,
 12,
 3,
 10,
 5,
 8,
 10,
 4,
 6,
 2,
 5,
 8,
 10]

In [10]:
#Generates Initial Random Seed For Simulation 
SeatingArrangementStep0=list(zip(GuestList, RandomArrangement))
SeatingArrangementStep0

[('Ms.Annikki Pietikäinen', 5),
 ('Ms.Line Kristiansen', 2),
 ('Mr.Koppány Baranyi', 5),
 ('Ms.Yukiko Koyama', 9),
 ('Ms.Johanna Wechsler', 9),
 ('Mr.Kristof Kuntz', 6),
 ('Ms.Živana Breceljnik', 7),
 ('Ms.Jitakatefuari ', 4),
 ('Mr.Bao Chin', 1),
 ('Mrs.Bikatu Bataev', 12),
 ('Mr.Teppo Lydman', 12),
 ('Ms.Adabella Quezada', 10),
 ('Ms.Apolonia Kucharska', 10),
 ('Ms.Lundy Moquin', 1),
 ('Mr.Jonathan Jessen', 5),
 ('Mrs.Jonna Parras', 12),
 ('Mr.Eduardo Santos', 4),
 ('Mr.Joao Ribeiro', 6),
 ('Mr.Meikuaritakufu ', 8),
 ('Mr.Géza Kovács', 11),
 ('Mr.Uzor Pál', 12),
 ('Mr.Þorgrímur Búason', 2),
 ('Mr.Shamsuddi Desheriyev', 11),
 ('Ms.Orzsebet Lendvai', 1),
 ('Mrs.Sarah Mortensen', 10),
 ('Ms.Miranda Zaragamba', 3),
 ('Mrs.Huan Yue Shen', 3),
 ('Mr.Cicerón Ozuna', 4),
 ('Mr.Shikimirikashite ', 2),
 ('Mr.Loren Pullman', 12),
 ('Mrs.Chinyere Onwuka', 6),
 ('Ms.Quinzia Schiavone', 8),
 ('Mr.Ross Morrison', 3),
 ('Mr.Yasser Hakimi', 7),
 ("Mr.Leal D'Aoust", 9),
 ('Dr.Hólmfríður Örlygsdóttir',

In [11]:
#As per above but ordered by Table No
SeatingArrangementSortedByTable = sorted(SeatingArrangementStep0, key = lambda x: x[1])
SeatingArrangementSortedByTable

[('Mr.Bao Chin', 1),
 ('Ms.Lundy Moquin', 1),
 ('Ms.Orzsebet Lendvai', 1),
 ('Mrs.Yobachi Chukwueloka', 1),
 ('Ms.Franziska Beich', 1),
 ('Mr.Justin Åberg', 1),
 ('Ms.Kyong Geraci', 1),
 ('Ms.Jitka Sobotková', 1),
 ('Ms.Line Kristiansen', 2),
 ('Mr.Þorgrímur Búason', 2),
 ('Mr.Shikimirikashite ', 2),
 ('Mrs.Tiina Johanson', 2),
 ('Mr.Marcel Lindholm', 2),
 ('Mr.Isaiah Tokaryev', 2),
 ('Mr.Marco Werner', 2),
 ('Spare Seat 1', 2),
 ('Ms.Miranda Zaragamba', 3),
 ('Mrs.Huan Yue Shen', 3),
 ('Mr.Ross Morrison', 3),
 ('Dr.Hólmfríður Örlygsdóttir', 3),
 ('Ms.Алла Евдокимова', 3),
 ('Mr.Johan Piekkanen', 3),
 ('Mrs.Renata Longo', 3),
 ('Mr.Trahand Burrows', 3),
 ('Ms.Jitakatefuari ', 4),
 ('Mr.Eduardo Santos', 4),
 ('Mr.Cicerón Ozuna', 4),
 ('Mrs.Zuzanna McIntosh', 4),
 ('Mr.Zoran Musa', 4),
 ('Mr.Corey Thomas', 4),
 ('Mrs.Justyna Wysocka', 4),
 ("Mrs.Yan Ch'in", 4),
 ('Ms.Annikki Pietikäinen', 5),
 ('Mr.Koppány Baranyi', 5),
 ('Mr.Jonathan Jessen', 5),
 ('Ms.Susanne Richter', 5),
 ('Mr.Carel 

In [12]:
# Translates list into a list of lists with tuples for Guest/Table No
SeatingArrangementSortedByTableChunked=[SeatingArrangementSortedByTable[i:i + int(table_size)] for i in range(0, len(SeatingArrangementSortedByTable), int(table_size))]
SeatingArrangementSortedByTableChunked

for i in range (0,int(noOfGuests/table_size)):
    print("Table No ",str(i+1))
    print(list(zip(*SeatingArrangementSortedByTableChunked[i]))[0])
    print(" ")

Table No  1
('Mr.Bao Chin', 'Ms.Lundy Moquin', 'Ms.Orzsebet Lendvai', 'Mrs.Yobachi Chukwueloka', 'Ms.Franziska Beich', 'Mr.Justin Åberg', 'Ms.Kyong Geraci', 'Ms.Jitka Sobotková')
 
Table No  2
('Ms.Line Kristiansen', 'Mr.Þorgrímur Búason', 'Mr.Shikimirikashite ', 'Mrs.Tiina Johanson', 'Mr.Marcel Lindholm', 'Mr.Isaiah Tokaryev', 'Mr.Marco Werner', 'Spare Seat 1')
 
Table No  3
('Ms.Miranda Zaragamba', 'Mrs.Huan Yue Shen', 'Mr.Ross Morrison', 'Dr.Hólmfríður Örlygsdóttir', 'Ms.Алла Евдокимова', 'Mr.Johan Piekkanen', 'Mrs.Renata Longo', 'Mr.Trahand Burrows')
 
Table No  4
('Ms.Jitakatefuari ', 'Mr.Eduardo Santos', 'Mr.Cicerón Ozuna', 'Mrs.Zuzanna McIntosh', 'Mr.Zoran Musa', 'Mr.Corey Thomas', 'Mrs.Justyna Wysocka', "Mrs.Yan Ch'in")
 
Table No  5
('Ms.Annikki Pietikäinen', 'Mr.Koppány Baranyi', 'Mr.Jonathan Jessen', 'Ms.Susanne Richter', 'Mr.Carel Voesten', 'Mr.Steffen Nacht', 'Ms.Zulfija Livk', 'Spare Seat 2')
 
Table No  6
('Mr.Kristof Kuntz', 'Mr.Joao Ribeiro', 'Mrs.Chinyere Onwuka', 'Mr

In [13]:
#Demonstration of a SINGLE ITERATION to show how conditions are tested

Score=0

for i in range (0,int(noOfGuests/table_size)):
    print(" ")
    print("Table No ",str(i+1))
    print("-------------------")
    print("Assigned Guests")
    print(list(zip(*SeatingArrangementSortedByTableChunked[i]))[0])
    for element in list(zip(*SeatingArrangementSortedByTableChunked[i]))[0]:
    #check if something is inside a list:
        
        if element in RelGuest:
            print(" ")
            print(element+" has conditions:")
            print(">Must be together with: ")
            print(RelConditionsTogether[RelGuest.index(element)])
            
            for j in range(0,3):
                try: 
                    if math.isnan(RelConditionsTogether[RelGuest.index(element)][j])==True:
                        print("PASS")
                        print("...")
                except TypeError:
                    if RelConditionsTogether[RelGuest.index(element)][j] in list(zip(*SeatingArrangementSortedByTableChunked[i]))[0]:    
                        print("Condition For "+element+" To Be Together With ")
                        print(RelConditionsTogether[RelGuest.index(element)][j])
                        print("PASS")
                        print("...")
                    else:
                        print("Condition For "+element+" To Be Together With ")
                        print(RelConditionsTogether[RelGuest.index(element)][j])
                        print("FAIL")
                        print("...")
                        Score += 1
                        
            print(">Must be apart from: ")
            
            print(RelConditionsApart[RelGuest.index(element)])
            
            for k in range(0,3):
                try: 
                    if math.isnan(RelConditionsApart[RelGuest.index(element)][k])==True:
                        print("PASS")
                        print("...")
                except TypeError:
                    if RelConditionsApart[RelGuest.index(element)][k] in list(zip(*SeatingArrangementSortedByTableChunked[i]))[0]:    
                        print("Condition For "+element+" To Be Apart From ")
                        print(RelConditionsApart[RelGuest.index(element)][k])
                        print("FAIL")
                        print("...")
                        Score += 1
                    else:
                        print("Condition For "+element+" To Be Apart From ")
                        print(RelConditionsApart[RelGuest.index(element)][k])
                        print("PASS")
                        print("...")
            
        else:
            print(" ")
            print(element+" has no conditions:")

print("------------------")
print("OVERALL SCORE:")
print(Score)
    


 
Table No  1
-------------------
Assigned Guests
('Mr.Bao Chin', 'Ms.Lundy Moquin', 'Ms.Orzsebet Lendvai', 'Mrs.Yobachi Chukwueloka', 'Ms.Franziska Beich', 'Mr.Justin Åberg', 'Ms.Kyong Geraci', 'Ms.Jitka Sobotková')
 
Mr.Bao Chin has conditions:
>Must be together with: 
['Mrs.Bao Chin', nan, nan]
Condition For Mr.Bao Chin To Be Together With 
Mrs.Bao Chin
FAIL
...
PASS
...
PASS
...
>Must be apart from: 
['Ms.Miranda Zaragamba', nan, nan]
Condition For Mr.Bao Chin To Be Apart From 
Ms.Miranda Zaragamba
PASS
...
PASS
...
PASS
...
 
Ms.Lundy Moquin has no conditions:
 
Ms.Orzsebet Lendvai has conditions:
>Must be together with: 
[nan, nan, nan]
PASS
...
PASS
...
PASS
...
>Must be apart from: 
['Mr.Shamsuddi Desheriyev', nan, nan]
Condition For Ms.Orzsebet Lendvai To Be Apart From 
Mr.Shamsuddi Desheriyev
PASS
...
PASS
...
PASS
...
 
Mrs.Yobachi Chukwueloka has no conditions:
 
Ms.Franziska Beich has no conditions:
 
Mr.Justin Åberg has no conditions:
 
Ms.Kyong Geraci has no conditions:


In [14]:
# RUN LOOPS TO X Iterations , Store Last "Best" Score
%time

maxIterations=10000

BestSoFar=100000 #Set to very high number to initialize it
BestSoFarArrangement=[]

for l in range(0,maxIterations):
    
    SeatingArrangementStep0=list(zip(GuestList, RandomArrangement))

    SeatingArrangementSortedByTable = sorted(SeatingArrangementStep0, key = lambda x: x[1])     

    SeatingArrangementSortedByTableChunked=[SeatingArrangementSortedByTable[i:i + int(table_size)] for i in range(0, len(SeatingArrangementSortedByTable), int(table_size))]


    Score=0
    
    for i in range (0,int(noOfGuests/table_size)):
        for element in list(zip(*SeatingArrangementSortedByTableChunked[i]))[0]:
        #check if something is inside a list:

            if element in RelGuest:
                for j in range(0,3):
                    try: 
                        if math.isnan(RelConditionsTogether[RelGuest.index(element)][j])==True:
                            pass
                    except TypeError:
                        if RelConditionsTogether[RelGuest.index(element)][j] in list(zip(*SeatingArrangementSortedByTableChunked[i]))[0]:    
                            pass
                        else:
                            Score += 1

                for k in range(0,3):
                    try: 
                        if math.isnan(RelConditionsApart[RelGuest.index(element)][k])==True:
                            pass
                    except TypeError:
                        if RelConditionsApart[RelGuest.index(element)][k] in list(zip(*SeatingArrangementSortedByTableChunked[i]))[0]:    
                            Score += 1
                        else:
                            pass
            else:
                pass
    
    print("Iteration",str(l),"Score=",str(Score)) 

    if Score<BestSoFar:
        BestSoFar=Score
        BestSoFarArrangement=RandomArrangement.copy()
    else:
        pass
    
    if Score>0:
        random.shuffle(RandomArrangement)
    else:
        break

Wall time: 0 ns
Iteration 0 Score= 8
Iteration 1 Score= 8
Iteration 2 Score= 7
Iteration 3 Score= 8
Iteration 4 Score= 7
Iteration 5 Score= 8
Iteration 6 Score= 7
Iteration 7 Score= 10
Iteration 8 Score= 8
Iteration 9 Score= 7
Iteration 10 Score= 9
Iteration 11 Score= 7
Iteration 12 Score= 8
Iteration 13 Score= 7
Iteration 14 Score= 7
Iteration 15 Score= 8
Iteration 16 Score= 10
Iteration 17 Score= 8
Iteration 18 Score= 7
Iteration 19 Score= 8
Iteration 20 Score= 9
Iteration 21 Score= 8
Iteration 22 Score= 8
Iteration 23 Score= 10
Iteration 24 Score= 6
Iteration 25 Score= 7
Iteration 26 Score= 8
Iteration 27 Score= 7
Iteration 28 Score= 7
Iteration 29 Score= 7
Iteration 30 Score= 7
Iteration 31 Score= 9
Iteration 32 Score= 8
Iteration 33 Score= 8
Iteration 34 Score= 8
Iteration 35 Score= 8
Iteration 36 Score= 7
Iteration 37 Score= 8
Iteration 38 Score= 8
Iteration 39 Score= 9
Iteration 40 Score= 8
Iteration 41 Score= 7
Iteration 42 Score= 10
Iteration 43 Score= 8
Iteration 44 Score= 9


Iteration 541 Score= 8
Iteration 542 Score= 8
Iteration 543 Score= 7
Iteration 544 Score= 10
Iteration 545 Score= 8
Iteration 546 Score= 7
Iteration 547 Score= 7
Iteration 548 Score= 7
Iteration 549 Score= 7
Iteration 550 Score= 10
Iteration 551 Score= 10
Iteration 552 Score= 7
Iteration 553 Score= 8
Iteration 554 Score= 8
Iteration 555 Score= 8
Iteration 556 Score= 10
Iteration 557 Score= 6
Iteration 558 Score= 7
Iteration 559 Score= 8
Iteration 560 Score= 8
Iteration 561 Score= 8
Iteration 562 Score= 8
Iteration 563 Score= 7
Iteration 564 Score= 7
Iteration 565 Score= 9
Iteration 566 Score= 7
Iteration 567 Score= 8
Iteration 568 Score= 8
Iteration 569 Score= 8
Iteration 570 Score= 8
Iteration 571 Score= 10
Iteration 572 Score= 7
Iteration 573 Score= 8
Iteration 574 Score= 7
Iteration 575 Score= 7
Iteration 576 Score= 7
Iteration 577 Score= 7
Iteration 578 Score= 7
Iteration 579 Score= 8
Iteration 580 Score= 7
Iteration 581 Score= 9
Iteration 582 Score= 12
Iteration 583 Score= 9
Itera

Iteration 895 Score= 10
Iteration 896 Score= 8
Iteration 897 Score= 6
Iteration 898 Score= 8
Iteration 899 Score= 12
Iteration 900 Score= 8
Iteration 901 Score= 8
Iteration 902 Score= 9
Iteration 903 Score= 10
Iteration 904 Score= 8
Iteration 905 Score= 8
Iteration 906 Score= 10
Iteration 907 Score= 8
Iteration 908 Score= 10
Iteration 909 Score= 4
Iteration 910 Score= 8
Iteration 911 Score= 8
Iteration 912 Score= 8
Iteration 913 Score= 6
Iteration 914 Score= 8
Iteration 915 Score= 9
Iteration 916 Score= 8
Iteration 917 Score= 8
Iteration 918 Score= 8
Iteration 919 Score= 8
Iteration 920 Score= 10
Iteration 921 Score= 8
Iteration 922 Score= 8
Iteration 923 Score= 7
Iteration 924 Score= 8
Iteration 925 Score= 7
Iteration 926 Score= 10
Iteration 927 Score= 7
Iteration 928 Score= 10
Iteration 929 Score= 9
Iteration 930 Score= 7
Iteration 931 Score= 8
Iteration 932 Score= 7
Iteration 933 Score= 8
Iteration 934 Score= 7
Iteration 935 Score= 10
Iteration 936 Score= 9
Iteration 937 Score= 8
It

Iteration 1378 Score= 7
Iteration 1379 Score= 8
Iteration 1380 Score= 7
Iteration 1381 Score= 10
Iteration 1382 Score= 8
Iteration 1383 Score= 7
Iteration 1384 Score= 8
Iteration 1385 Score= 7
Iteration 1386 Score= 7
Iteration 1387 Score= 8
Iteration 1388 Score= 8
Iteration 1389 Score= 8
Iteration 1390 Score= 7
Iteration 1391 Score= 8
Iteration 1392 Score= 7
Iteration 1393 Score= 8
Iteration 1394 Score= 7
Iteration 1395 Score= 7
Iteration 1396 Score= 8
Iteration 1397 Score= 10
Iteration 1398 Score= 8
Iteration 1399 Score= 7
Iteration 1400 Score= 6
Iteration 1401 Score= 8
Iteration 1402 Score= 9
Iteration 1403 Score= 10
Iteration 1404 Score= 7
Iteration 1405 Score= 8
Iteration 1406 Score= 7
Iteration 1407 Score= 8
Iteration 1408 Score= 10
Iteration 1409 Score= 8
Iteration 1410 Score= 7
Iteration 1411 Score= 6
Iteration 1412 Score= 8
Iteration 1413 Score= 7
Iteration 1414 Score= 6
Iteration 1415 Score= 10
Iteration 1416 Score= 7
Iteration 1417 Score= 10
Iteration 1418 Score= 7
Iteration 

Iteration 1782 Score= 7
Iteration 1783 Score= 7
Iteration 1784 Score= 7
Iteration 1785 Score= 6
Iteration 1786 Score= 8
Iteration 1787 Score= 7
Iteration 1788 Score= 6
Iteration 1789 Score= 7
Iteration 1790 Score= 8
Iteration 1791 Score= 7
Iteration 1792 Score= 7
Iteration 1793 Score= 6
Iteration 1794 Score= 8
Iteration 1795 Score= 8
Iteration 1796 Score= 7
Iteration 1797 Score= 7
Iteration 1798 Score= 10
Iteration 1799 Score= 6
Iteration 1800 Score= 8
Iteration 1801 Score= 10
Iteration 1802 Score= 8
Iteration 1803 Score= 8
Iteration 1804 Score= 8
Iteration 1805 Score= 7
Iteration 1806 Score= 7
Iteration 1807 Score= 8
Iteration 1808 Score= 10
Iteration 1809 Score= 6
Iteration 1810 Score= 8
Iteration 1811 Score= 7
Iteration 1812 Score= 7
Iteration 1813 Score= 10
Iteration 1814 Score= 8
Iteration 1815 Score= 9
Iteration 1816 Score= 7
Iteration 1817 Score= 7
Iteration 1818 Score= 7
Iteration 1819 Score= 8
Iteration 1820 Score= 7
Iteration 1821 Score= 6
Iteration 1822 Score= 8
Iteration 18

...

Iteration 8107 Score= 7
Iteration 8108 Score= 8
Iteration 8109 Score= 7
Iteration 8110 Score= 9
Iteration 8111 Score= 8
Iteration 8112 Score= 7
Iteration 8113 Score= 10
Iteration 8114 Score= 7
Iteration 8115 Score= 7
Iteration 8116 Score= 8
Iteration 8117 Score= 7
Iteration 8118 Score= 7
Iteration 8119 Score= 10
Iteration 8120 Score= 8
Iteration 8121 Score= 7
Iteration 8122 Score= 8
Iteration 8123 Score= 7
Iteration 8124 Score= 9
Iteration 8125 Score= 7
Iteration 8126 Score= 7
Iteration 8127 Score= 6
Iteration 8128 Score= 8
Iteration 8129 Score= 8
Iteration 8130 Score= 10
Iteration 8131 Score= 8
Iteration 8132 Score= 6
Iteration 8133 Score= 8
Iteration 8134 Score= 8
Iteration 8135 Score= 8
Iteration 8136 Score= 7
Iteration 8137 Score= 9
Iteration 8138 Score= 8
Iteration 8139 Score= 8
Iteration 8140 Score= 8
Iteration 8141 Score= 9
Iteration 8142 Score= 8
Iteration 8143 Score= 8
Iteration 8144 Score= 10
Iteration 8145 Score= 8
Iteration 8146 Score= 6
Iteration 8147 Score= 8
Iteration 81

Iteration 8606 Score= 8
Iteration 8607 Score= 10
Iteration 8608 Score= 9
Iteration 8609 Score= 7
Iteration 8610 Score= 8
Iteration 8611 Score= 8
Iteration 8612 Score= 9
Iteration 8613 Score= 7
Iteration 8614 Score= 7
Iteration 8615 Score= 7
Iteration 8616 Score= 10
Iteration 8617 Score= 6
Iteration 8618 Score= 8
Iteration 8619 Score= 7
Iteration 8620 Score= 8
Iteration 8621 Score= 8
Iteration 8622 Score= 7
Iteration 8623 Score= 7
Iteration 8624 Score= 8
Iteration 8625 Score= 10
Iteration 8626 Score= 7
Iteration 8627 Score= 7
Iteration 8628 Score= 10
Iteration 8629 Score= 10
Iteration 8630 Score= 10
Iteration 8631 Score= 7
Iteration 8632 Score= 8
Iteration 8633 Score= 6
Iteration 8634 Score= 7
Iteration 8635 Score= 7
Iteration 8636 Score= 8
Iteration 8637 Score= 10
Iteration 8638 Score= 8
Iteration 8639 Score= 8
Iteration 8640 Score= 8
Iteration 8641 Score= 7
Iteration 8642 Score= 8
Iteration 8643 Score= 8
Iteration 8644 Score= 7
Iteration 8645 Score= 8
Iteration 8646 Score= 7
Iteration

Iteration 8956 Score= 10
Iteration 8957 Score= 7
Iteration 8958 Score= 6
Iteration 8959 Score= 8
Iteration 8960 Score= 7
Iteration 8961 Score= 10
Iteration 8962 Score= 10
Iteration 8963 Score= 8
Iteration 8964 Score= 8
Iteration 8965 Score= 10
Iteration 8966 Score= 8
Iteration 8967 Score= 9
Iteration 8968 Score= 7
Iteration 8969 Score= 7
Iteration 8970 Score= 11
Iteration 8971 Score= 7
Iteration 8972 Score= 8
Iteration 8973 Score= 6
Iteration 8974 Score= 8
Iteration 8975 Score= 8
Iteration 8976 Score= 8
Iteration 8977 Score= 8
Iteration 8978 Score= 8
Iteration 8979 Score= 6
Iteration 8980 Score= 7
Iteration 8981 Score= 8
Iteration 8982 Score= 7
Iteration 8983 Score= 10
Iteration 8984 Score= 9
Iteration 8985 Score= 7
Iteration 8986 Score= 7
Iteration 8987 Score= 9
Iteration 8988 Score= 8
Iteration 8989 Score= 8
Iteration 8990 Score= 6
Iteration 8991 Score= 8
Iteration 8992 Score= 10
Iteration 8993 Score= 10
Iteration 8994 Score= 10
Iteration 8995 Score= 10
Iteration 8996 Score= 10
Itera

Iteration 9309 Score= 8
Iteration 9310 Score= 6
Iteration 9311 Score= 8
Iteration 9312 Score= 7
Iteration 9313 Score= 8
Iteration 9314 Score= 7
Iteration 9315 Score= 9
Iteration 9316 Score= 7
Iteration 9317 Score= 8
Iteration 9318 Score= 8
Iteration 9319 Score= 8
Iteration 9320 Score= 8
Iteration 9321 Score= 5
Iteration 9322 Score= 8
Iteration 9323 Score= 7
Iteration 9324 Score= 8
Iteration 9325 Score= 9
Iteration 9326 Score= 8
Iteration 9327 Score= 7
Iteration 9328 Score= 7
Iteration 9329 Score= 7
Iteration 9330 Score= 8
Iteration 9331 Score= 7
Iteration 9332 Score= 5
Iteration 9333 Score= 10
Iteration 9334 Score= 8
Iteration 9335 Score= 7
Iteration 9336 Score= 7
Iteration 9337 Score= 7
Iteration 9338 Score= 8
Iteration 9339 Score= 8
Iteration 9340 Score= 7
Iteration 9341 Score= 7
Iteration 9342 Score= 8
Iteration 9343 Score= 10
Iteration 9344 Score= 8
Iteration 9345 Score= 9
Iteration 9346 Score= 8
Iteration 9347 Score= 8
Iteration 9348 Score= 7
Iteration 9349 Score= 8
Iteration 9350

Iteration 9780 Score= 9
Iteration 9781 Score= 9
Iteration 9782 Score= 8
Iteration 9783 Score= 8
Iteration 9784 Score= 7
Iteration 9785 Score= 8
Iteration 9786 Score= 7
Iteration 9787 Score= 8
Iteration 9788 Score= 7
Iteration 9789 Score= 7
Iteration 9790 Score= 11
Iteration 9791 Score= 7
Iteration 9792 Score= 7
Iteration 9793 Score= 8
Iteration 9794 Score= 10
Iteration 9795 Score= 10
Iteration 9796 Score= 10
Iteration 9797 Score= 8
Iteration 9798 Score= 8
Iteration 9799 Score= 8
Iteration 9800 Score= 8
Iteration 9801 Score= 7
Iteration 9802 Score= 9
Iteration 9803 Score= 8
Iteration 9804 Score= 7
Iteration 9805 Score= 8
Iteration 9806 Score= 8
Iteration 9807 Score= 8
Iteration 9808 Score= 8
Iteration 9809 Score= 8
Iteration 9810 Score= 10
Iteration 9811 Score= 8
Iteration 9812 Score= 8
Iteration 9813 Score= 8
Iteration 9814 Score= 7
Iteration 9815 Score= 8
Iteration 9816 Score= 6
Iteration 9817 Score= 7
Iteration 9818 Score= 10
Iteration 9819 Score= 8
Iteration 9820 Score= 10
Iteration

In [15]:
if Score!=0:
    print(" ")
    print("WARNING : Unable To Find Ideal Solution With Score=0")
    print("Best Score Out Of:",str(maxIterations),"Random Iterations Was Score=",str(BestSoFar))
    print("Best Arrangement Found : ")
    print(" ")
    BestSoFarSeatingArrangement=list(zip(GuestList, RandomArrangement))
    
    BestSoFarSeatingArrangementSortedByTable = sorted(BestSoFarSeatingArrangement, key = lambda x: x[1])
    BestSoFarSeatingArrangementSortedByTableChunked=[BestSoFarSeatingArrangementSortedByTable[i:i + int(table_size)] for i in range(0, len(BestSoFarSeatingArrangementSortedByTable), int(table_size))]
    
    FinalSeatingArrangementSortedByTableFlat = functools.reduce(operator.iconcat, BestSoFarSeatingArrangementSortedByTableChunked, [])

elif Score==0:
    print(" ")
    print("SUCCESS: Found Ideal Solution With Score=0")
    print("Ideal Arrangement Found : ")
    print(" ")
    FinalSeatingArrangementSortedByTableFlat = functools.reduce(operator.iconcat, SeatingArrangementSortedByTableChunked, [])

FinalSeatingArrangementSortedByTableUnzipped = zip(*FinalSeatingArrangementSortedByTableFlat)
FinalSeatingArrangementSortedByTableUnzippedList = list(FinalSeatingArrangementSortedByTableUnzipped)
FinalSeatingArrangementDF=pd.DataFrame(FinalSeatingArrangementSortedByTableUnzippedList).T
FinalSeatingArrangementDF.columns=["Guest","Assigned Table"]
    
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(FinalSeatingArrangementDF)

 
Best Score Out Of: 10000 Random Iterations Was Score= 3
Best Arrangement Found : 
 
                         Guest Assigned Table
0           Mr.Koppány Baranyi              1
1             Mr.Yasser Hakimi              1
2   Dr.Hólmfríður Örlygsdóttir              1
3           Ms.Franziska Beich              1
4             Ms.Bahijah Gaber              1
5            Ms.Laura McDonald              1
6              Mr.Marco Werner              1
7           Mr.Andreas Bruheim              1
8          Mrs.Sarah Mortensen              2
9             Mr.Steffen Nacht              2
10           Ms.Christin Meyer              2
11          Mr.Isaiah Tokaryev              2
12          Mr.Fletcher Boivin              2
13             Mrs.Rhea Gibson              2
14             Mr.Justin Åberg              2
15         Mrs.Justyna Wysocka              2
16         Ms.Johanna Wechsler              3
17          Ms.Алла Евдокимова              3
18      Ms.Ebelechukwu Chineze          

In [16]:
FinalSeatingArrangementWConditions = pd.merge(FinalSeatingArrangementDF,RelMatrixRaw,on ='Guest',how='left')

FinalSeatingArrangementWConditions =FinalSeatingArrangementWConditions[["Guest","Assigned Table","Together1","Together2","Together3","Apart1","Apart2","Apart3"]]

FinalSeatingArrangementWConditions 

,Guest,Assigned Table,Together1,Together2,Together3,Apart1,Apart2,Apart3
0,Mr.Koppány Baranyi,1,NaN,NaN,NaN,NaN,NaN,NaN
1,Mr.Yasser Hakimi,1,NaN,NaN,NaN,Dr.Hólmfríður Örlygsdóttir,NaN,NaN
2,Dr.Hólmfríður Örlygsdóttir,1,NaN,NaN,NaN,Mr.Yasser Hakimi,NaN,NaN
3,Ms.Franziska Beich,1,NaN,NaN,NaN,NaN,NaN,NaN
4,Ms.Bahijah Gaber,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
91,Mr.Loren Pullman,12,NaN,NaN,NaN,NaN,NaN,NaN
92,Mr.Carel Voesten,12,NaN,NaN,NaN,NaN,NaN,NaN
93,Mr.Metody Symanski,12,NaN,NaN,NaN,NaN,NaN,NaN
94,Mr.Zygmunt Wiśniewski,12,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#EXPORT RESULTS TO CSV
FinalSeatingArrangementWConditions.to_csv("Output_BruteForce_"+InputFileName.strip('.xls')+".csv")

In [ ]:
#Optional Code - Generates Relationship Edges for Use With Other Notebook That Uses Simulated Annealing

# RelMatrixRaw
# relationships_edges={}
# Together1=RelMatrixRaw[["Guest","Together1"]].dropna(thresh=2)
# Together2=RelMatrixRaw[["Guest","Together2"]].dropna(thresh=2)
# Together3=RelMatrixRaw[["Guest","Together3"]].dropna(thresh=2)
# Together1.columns=["Guest","Together"]
# Together2.columns=["Guest","Together"]
# Together3.columns=["Guest","Together"]
# Together=pd.concat([Together1,Together2,Together3])
# Apart1=RelMatrixRaw[["Guest","Apart1"]].dropna(thresh=2)
# Apart2=RelMatrixRaw[["Guest","Apart2"]].dropna(thresh=2)
# Apart3=RelMatrixRaw[["Guest","Apart3"]].dropna(thresh=2)
# Apart1.columns=["Guest","Apart"]
# Apart2.columns=["Guest","Apart"]
# Apart3.columns=["Guest","Apart"]
# Apart=pd.concat([Apart1,Apart2,Apart3])
# for element in list(zip(Together["Guest"], Together["Together"])):
#         relationships_edges.update({element:-50})
# for element in list(zip(Apart["Guest"], Apart["Apart"])):
#         relationships_edges.update({element:50})                
# relationships_edges